In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('max_columns', 300)

kfolds = KFold(n_splits=5, shuffle=True, random_state=345)

In [2]:
df = pd.read_csv('data/processed_men/training_data.csv')

df.head()

,Season,DayNum,Team1,Team2,target,target_points,T1_Loc,T2_Loc,ID,T1_Score,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T1_FGM2,T1_FGA2,T1_Tot_Reb,T1_FGM_no_ast,T1_possessions,T1_off_rating,T1_shtg_opportunity,T1_TO_perposs,T1_def_rating,T1_opp_shtg_opportunity,T1_opp_possessions,T1_opp_score,T1_opp_FTA,T1_opp_FGA,T1_impact,T1_DR_opportunity,T1_OR_opportunity,T1_Score_diff,T1_Score_advantage,T1_FGM_diff,T1_FGM_advantage,T1_FGA_diff,T1_FGA_advantage,T1_FGM3_diff,T1_FGM3_advantage,T1_FGA3_diff,T1_FGA3_advantage,T1_FTM_diff,T1_FTM_advantage,T1_FTA_diff,T1_FTA_advantage,T1_OR_diff,T1_OR_advantage,T1_DR_diff,T1_DR_advantage,T1_Ast_diff,T1_Ast_advantage,T1_TO_diff,T1_TO_advantage,T1_Stl_diff,T1_Stl_advantage,T1_Blk_diff,T1_Blk_advantage,T1_PF_diff,T1_PF_advantage,T1_FGM2_diff,T1_FGM2_advantage,T1_FGA2_diff,T1_FGA2_advantage,T1_Tot_Reb_diff,T1_Tot_Reb_advantage,T1_FGM_no_ast_diff,T1_FGM_no_ast_advantage,T1_DR_opportunity_diff,T1_DR_opportunity_advantage,T1_OR_opportunity_diff,T1_OR_opportunity_advantage,T1_possessions_diff,T1_possessions_advantage,T1_off_rating_diff,T1_off_rating_advantage,T1_def_rating_diff,T1_def_rating_advantage,T1_shtg_opportunity_diff,T1_shtg_opportunity_advantage,T1_TO_perposs_diff,T1_TO_perposs_advantage,T1_impact_diff,T1_impact_advantage,T1_True_shooting_perc_diff,T1_True_shooting_perc_advantage,T1_N_wins,T1_FGM_perc,T1_FGM2_perc,T1_FGM3_perc,T1_FT_perc,T1_FGM_no_ast_perc,T1_True_shooting_perc,T1_Opp_True_shooting_perc,T1_Seed,T2_Score,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,T2_FGM2,T2_FGA2,T2_Tot_Reb,T2_FGM_no_ast,T2_possessions,T2_off_rating,T2_shtg_opportunity,T2_TO_perposs,T2_def_rating,T2_opp_shtg_opportunity,T2_opp_possessions,T2_opp_score,T2_opp_FTA,T2_opp_FGA,T2_impact,T2_DR_opportunity,T2_OR_opportunity,T2_Score_diff,T2_Score_advantage,T2_FGM_diff,T2_FGM_advantage,T2_FGA_diff,T2_FGA_advantage,T2_FGM3_diff,T2_FGM3_advantage,T2_FGA3_diff,T2_FGA3_advantage,T2_FTM_diff,T2_FTM_advantage,T2_FTA_diff,T2_FTA_advantage,T2_OR_diff,T2_OR_advantage,T2_DR_diff,T2_DR_advantage,T2_Ast_diff,T2_Ast_advantage,T2_TO_diff,T2_TO_advantage,T2_Stl_diff,T2_Stl_advantage,T2_Blk_diff,T2_Blk_advantage,T2_PF_diff,T2_PF_advantage,T2_FGM2_diff,T2_FGM2_advantage,T2_FGA2_diff,T2_FGA2_advantage,T2_Tot_Reb_diff,T2_Tot_Reb_advantage,T2_FGM_no_ast_diff,T2_FGM_no_ast_advantage,T2_DR_opportunity_diff,T2_DR_opportunity_advantage,T2_OR_opportunity_diff,T2_OR_opportunity_advantage,T2_possessions_diff,T2_possessions_advantage,T2_off_rating_diff,T2_off_rating_advantage,T2_def_rating_diff,T2_def_rating_advantage,T2_shtg_opportunity_diff,T2_shtg_opportunity_advantage,T2_TO_perposs_diff,T2_TO_perposs_advantage,T2_impact_diff,T2_impact_advantage,T2_True_shooting_perc_diff,T2_True_shooting_perc_advantage,T2_N_wins,T2_FGM_perc,T2_FGM2_perc,T2_FGM3_perc,T2_FT_perc,T2_FGM_no_ast_perc,T2_True_shooting_perc,T2_Opp_True_shooting_perc,T2_Seed,delta_Loc,delta_Score,delta_FGM,delta_FGA,delta_FGM3,delta_FGA3,delta_FTM,delta_FTA,delta_OR,delta_DR,delta_Ast,delta_TO,delta_Stl,delta_Blk,delta_PF,delta_FGM2,delta_FGA2,delta_Tot_Reb,delta_FGM_no_ast,delta_possessions,delta_off_rating,delta_shtg_opportunity,delta_TO_perposs,delta_def_rating,delta_opp_shtg_opportunity,delta_opp_possessions,delta_opp_score,delta_opp_FTA,delta_opp_FGA,delta_impact,delta_DR_opportunity,delta_OR_opportunity,delta_Score_diff,delta_Score_advantage,delta_FGM_diff,delta_FGM_advantage,delta_FGA_diff,delta_FGA_advantage,delta_FGM3_diff,delta_FGM3_advantage,delta_FGA3_diff,delta_FGA3_advantage,delta_FTM_diff,delta_FTM_advantage,delta_FTA_diff,delta_FTA_advantage,delta_OR_diff,delta_OR_advantage,delta_DR_diff,delta_DR_advantage,delta_Ast_diff,delta_Ast_advantage,delta_TO_diff,delta_TO_advantage,delta_Stl_diff,delta_Stl_advantage,delta_Blk_diff,delta_Blk_advantage,delta_PF_diff,delta_PF_advantage,delta_FGM2_diff,delta_FGM2_advantage,delta_FGA2_diff,delta_FGA2_advantage,delta_Tot_Reb_diff,delta_Tot_Reb_a

In [3]:
ranks = pd.read_csv('data/raw_men/MDataFiles_Stage1/MMasseyOrdinals.csv')

ranks.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [4]:
ranks.SystemName.nunique()

174

In [6]:
ranks.groupby(['Season', 'SystemName']).OrdinalRank.agg(['min', 'max', 'mean']).describe()

,min,max,mean
count,945.0,945.000000,945.000000
mean,1.0,324.065608,162.367317
std,0.0,77.445005,38.736448
min,1.0,25.000000,12.978947
25%,1.0,336.000000,168.500000
50%,1.0,347.000000,174.000000
75%,1.0,351.000000,176.000000
max,1.0,353.000000,177.000000


In [13]:
tmp = ranks.groupby(['Season', 'SystemName']).OrdinalRank.agg(['min', 'max', 'mean']).reset_index()

tmp[tmp['max'] < 300].SystemName.unique()

array(['AP', 'USA', 'DES', 'LYN', 'ACU', 'TRX', 'D1A', 'JNG', 'BNT'],
      dtype=object)

In [17]:
ranks = ranks[~(ranks.SystemName.isin(['AP', 'USA', 'DES', 'LYN', 'ACU', 'TRX', 'D1A', 'JNG', 'BNT']))].copy()

ranks.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [19]:
ranks.groupby(['Season', 'SystemName']).OrdinalRank.agg(['min', 'max', 'mean']).describe()

,min,max,mean
count,869.0,869.000000,869.000000
mean,1.0,344.845800,172.780662
std,0.0,8.063925,4.110886
min,1.0,324.000000,159.988095
25%,1.0,341.000000,171.000000
50%,1.0,347.000000,174.000000
75%,1.0,351.000000,176.000000
max,1.0,353.000000,177.000000


In [30]:
regular_season = 'data/raw_men/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv'

reg = pd.read_csv(regular_season)

reg.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [31]:
mean_ranks = ranks.groupby(['Season', 'TeamID', 'RankingDayNum'], as_index=False).OrdinalRank.mean()

mean_ranks.sample(10)

,Season,TeamID,RankingDayNum,OrdinalRank
29838,2004,1425,43,138.700000
27924,2004,1386,91,2.407407
83939,2008,1251,79,203.000000
153401,2012,1201,106,194.733333
195275,2015,1359,30,288.923077
169128,2013,1160,84,38.052632
3878,2003,1194,69,259.000000
178530,2013,1351,128,163.700000
59835,2006,1355,127,268.500000
79499,2008,1155,121,23.000000


In [32]:
reg = pd.merge(reg, mean_ranks.rename(columns={'TeamID': 'WTeamID', 
                                                'RankingDayNum':'DayNum', 
                                                'OrdinalRank': 'WRank'}), 
                on=['Season', 'WTeamID', 'DayNum'], how='left')

reg = pd.merge(reg, mean_ranks.rename(columns={'TeamID': 'LTeamID', 
                                                'RankingDayNum':'DayNum', 
                                                'OrdinalRank': 'LRank'}), 
                on=['Season', 'LTeamID', 'DayNum'], how='left')

reg = reg.fillna(1000)

reg.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WRank,LRank
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,1000.0,1000.0
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1000.0,1000.0
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,1000.0,1000.0
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,1000.0,1000.0
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,1000.0,1000.0


In [37]:
reg['top_team_win'] = 0
reg.loc[reg.LRank <= 30, 'top_team_win'] = 1

reg.top_team_win.mean()

0.010788078259279577

In [41]:
reg['upset'] = 0
reg.loc[reg.WRank - reg.LRank > 15, 'upset'] = 1

reg.upset.mean()

0.05493463155970013

In [42]:
reg['WAway'] = 0
reg.loc[reg.WLoc!='H', 'WAway'] = 1

reg.WAway.mean()

0.4077413603949534

In [44]:
ot_wins = reg[['Season', 'WTeamID', 'NumOT']].copy()

ot_wins['has_OT'] = np.where(ot_wins.NumOT > 0, 1, 0)
ot_wins['OT_win'] = np.where(ot_wins.has_OT > 0, 1, 0)

ot_wins.head()

,Season,WTeamID,NumOT,has_OT,OT_win
0,2003,1104,0,0,0
1,2003,1272,0,0,0
2,2003,1266,0,0,0
3,2003,1296,0,0,0
4,2003,1400,0,0,0


In [45]:
ot_wins = ot_wins.groupby(['Season', 'WTeamID', 'has_OT'], as_index=False).OT_win.mean()

ot_wins.sample(10)

,Season,WTeamID,has_OT,OT_win
7745,2017,1222,0,0
2635,2008,1153,1,1
8237,2018,1170,0,0
892,2004,1391,0,0
8807,2019,1202,0,0
8527,2018,1351,0,0
583,2004,1162,1,1
2624,2008,1145,0,0
1252,2005,1281,1,1
6085,2014,1256,1,1


In [47]:
ot_wins[ot_wins.has_OT==1].describe()

,Season,WTeamID,has_OT,OT_win
count,3246.000000,3246.000000,3246.0,3246.0
mean,2010.879544,1285.123537,1.0,1.0
std,4.582337,104.431833,0.0,0.0
min,2003.000000,1101.000000,1.0,1.0
25%,2007.000000,1196.000000,1.0,1.0
50%,2011.000000,1282.500000,1.0,1.0
75%,2015.000000,1377.000000,1.0,1.0
max,2019.000000,1464.000000,1.0,1.0
